In [97]:
## Standard libraries
import os
import json
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms

/tmp/ipykernel_889/3837100366.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [98]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

DATASET_PATH = "../data"
CHECKPOINT_PATH = "../saved_models/tutorial7"

pl.seed_everything(42)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

print(f"Using device {device}")

Global seed set to 42


Using device cuda:0


In [99]:
import urllib.request
from urllib.error import HTTPError
# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/"
# Files to download
pretrained_files = ["NodeLevelMLP.ckpt", "NodeLevelGNN.ckpt", "GraphLevelGraphConv.ckpt"]

# Create checkpoint path if it doesn't exist yet
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# For each file, check whether it already exists. If not, try downloading it.
for file_name in pretrained_files:
    file_path = os.path.join(CHECKPOINT_PATH, file_name)
    if "/" in file_name:
        os.makedirs(file_path.rsplit("/",1)[0], exist_ok=True)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Downloading {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

## Graph Neural Networks

An **adjacency matrix** is a square matrix whose elements indicate whether pairs of vertices are adjacent (connected or not). $A_{ij}$ is 1 if there is a connection from node $i$ to $j$. FOr an undirected graph, $A$ is a symmetric matrix.

### Graph Convolutions

GCNs (Graph Convolution Networks) are similar to convolutions in images in the sense that the “filter” parameters are typically shared over all locations in the graph. At the same time, GCNs rely on message passing methods, which means that vertices exchange information with the neighbors, and send “messages” to each other. <br>

The first step is that each node creates a feature vector that represents the message that it wants to send. The second step is messages are sent to neighbors so that a node receives one message per adjacent node. <br>

An arbritrary number of messages need to be combined in some way for a node to receive them. The usual way to go is to sum or take the mean.

In [100]:
class GCNLayer(nn.Module):
    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out) # convert input features to messages

    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch, num_nodes, c_in]
            adj_matrix - batch of adj matrices of the graph. If there is an edge from i to j,
            adj_matrix[b,i,j] = 1 else 0. Supports directed edges by non-symmetric matrices. Assume
            to already have added the identity connections (A = A + I since each messages gets 
            message from itself)
        """
        num_neighbors = adj_matrix.sum(dim=-1, keepdims=True) # get num neighbors for each node
                                                              # include itself from A = A + I
        node_feats = self.projection(node_feats)
        node_feats = torch.bmm(adj_matrix, node_feats)
        node_feats = node_feats / num_neighbors # average
        return node_feats

In [101]:
node_feats = torch.arange(8, dtype=torch.float32).view(1,4,2)
adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                            [1, 1, 1, 1],
                            [0, 1, 1, 1],
                            [0, 1, 1, 1]]])

print("Node features:\n", node_feats)
print("\nAdjacency matrix:\n", adj_matrix)

Node features:
 tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Adjacency matrix:
 tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])


In [102]:
# apply GCN layer to above example

gcn_layer = GCNLayer(c_in=2, c_out=4)
gcn_layer.projection.weight.data = torch.Tensor([[1.0, 0.0],[0.0, 1.0]])
gcn_layer.projection.bias.data = torch.Tensor([0.0, 0.0])

with torch.no_grad():
    out_feats = gcn_layer(node_feats, adj_matrix)

print("Output features: \n", out_feats)

Output features: 
 tensor([[[1., 2.],
         [3., 4.],
         [4., 5.],
         [4., 5.]]])


Can see from above that each node's output features are just the average of the summed self and neighbor values. In a GNN, we also want feature exchange between nodes beyond its neighbors. This can be achieved by applying multiple GCN layers. However, one issue we can see from the above example is the output features of 3 and 4 are the same since they have the same adjacent nodes (inclusive of self). Therefore, the GCN layer can make the network forget node-specific info if we just take a mean over all messages.

#### Aside: Einsum

[Tutorial](https://rockt.github.io/2018/04/30/einsum) here.
[Blog](https://obilaniu6266h16.wordpress.com/2016/02/04/einstein-summation-in-numpy/) here.

Einsum notation is an elegant way to express ot products, outer products, transposes and matrix-vector or matrix-matrix multiplications. Once you understand and make use of einsum, you will be able to write more concise and efficient code more quickly. When not using einsum it is easy to introduce unnecessary reshaping and transposing of tensors, as well as intermediate tensors that could be omitted.

In [103]:
# Matrix transpose: B_{ji} = A_{ij}
a = torch.arange(6).view(2,3)
a_T = torch.einsum("ij->ji", [a])
print("a: \n", a)
print("a.T: \n", a_T)

a: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
a.T: 
 tensor([[0, 3],
        [1, 4],
        [2, 5]])


In [104]:
# Sum
a = torch.arange(6).view(2,3)
a_sum = torch.einsum("ij->", [a])
print("a: \n", a)
print("a_sum: \n", a_sum)

a: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
a_sum: 
 tensor(15)


In [105]:
# Column sum
a = torch.arange(6).view(2,3)
a_col = torch.einsum("ij->i", [a])
print("a: \n", a)
print("a_col: \n", a_col)

a: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
a_col: 
 tensor([ 3, 12])


In [106]:
# Row sum
a = torch.arange(6).view(2,3)
a_row = torch.einsum("ij->j", [a])
print("a: \n", a)
print("a_row: \n", a_row)

a: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
a_row: 
 tensor([3, 5, 7])


In [107]:
# matrix-vector multiplication
a = torch.arange(6).view(2,3)
b = torch.arange(3)
ab = torch.einsum("ik,k->i",[a,b])
print("a: \n", a)
print("b: \n", b)
print("a@b: \n", ab)

a: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
b: 
 tensor([0, 1, 2])
a@b: 
 tensor([ 5, 14])


In [108]:
# matrix-matrix multiplication
a = torch.arange(6).view(2,3)
b = torch.arange(15).view(3,5)
ab = torch.einsum("ij,jk->ik",[a,b])
print("a: \n", a)
print("b: \n", b)
print("a@b: \n", ab)

a: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
b: 
 tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14]])
a@b: 
 tensor([[ 25,  28,  31,  34,  37],
        [ 70,  82,  94, 106, 118]])


In [119]:
# dot product - vector
a = torch.arange(3)
b = torch.arange(3,6)
a_dot_b = torch.einsum("i,j->", [a,b])
print("a: \n", a)
print("b: \n", b)
print("a dot b: \n", a_dot_b)

a: 
 tensor([0, 1, 2])
b: 
 tensor([3, 4, 5])
a dot b: 
 tensor(36)


In [110]:
# dot product - matrix
a = torch.arange(6).view(2,3)
b = torch.arange(6,12).view(2,3)
a_dot_b = torch.einsum("ij,ij->", [a,b])
print("a: \n", a)
print("b: \n", b)
print("a dot b: \n", a_dot_b)

a: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
b: 
 tensor([[ 6,  7,  8],
        [ 9, 10, 11]])
a dot b: 
 tensor(145)


In [111]:
# Hadamard Product (element wise multiplication)
a = torch.arange(6).view(2,3)
b = torch.arange(6,12).view(2,3)
a_dot_b = torch.einsum("ij,ij->ij",[a,b])
print("a: \n", a)
print("b: \n", b)
print("a dot b: \n", a_dot_b)

a: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
b: 
 tensor([[ 6,  7,  8],
        [ 9, 10, 11]])
a dot b: 
 tensor([[ 0,  7, 16],
        [27, 40, 55]])


In [112]:
# outer product
a = torch.arange(3)
b = torch.arange(3,7)
ab_out = torch.einsum("i,j->ij",[a,b])
print("a: \n", a)
print("b: \n", b)
print("a outer b: \n", ab_out)

a: 
 tensor([0, 1, 2])
b: 
 tensor([3, 4, 5, 6])
a outer b: 
 tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]])


In [113]:
# batch matrix multiplication
a = torch.randn(3,2,5)
b = torch.randn(3,5,3)
torch.einsum("ijk,ikl->ijl",[a,b])

tensor([[[-1.8168, -2.7787,  0.9705],
         [ 1.1204,  1.6555,  0.7336]],

        [[ 4.3613, -0.6342, -2.4094],
         [ 0.7059, -1.4787,  1.1062]],

        [[ 0.8910,  0.7363,  5.5488],
         [ 1.7569,  0.0972, -0.3498]]])

In [114]:
# bilinear transformation
a = torch.randn(2,3)
b = torch.randn(5,3,7)
c = torch.randn(2,7)
torch.einsum("ik,jkl,il->ij",[a,b,c])

tensor([[-0.7972,  2.7966, -1.6671,  2.2419, -3.6788],
        [ 1.3286, -7.7852,  1.5088, -2.0807,  2.0390]])

### Graph Attention

Similarly to the GCN, the graph attention layer creates a message for each node using a linear layer/weight matrix. For the attention part, it uses the message from the node itself as a query, and the messages to average as both keys and values (note that this also includes the message to itself). The score function $f_{attn}$ is implemented as a one-layer MLP which maps the query and key to a single value.

In [115]:
class GATLayer(nn.Module):
    def __init__(self, c_in, c_out, num_heads=1, concat_heads=True, alpha=0.2):
        """
        Inputs:
            c_in - Dimensionality of input features
            c_out - Dimensionality of output features
            num_heads - Number of heads, i.e. attention mechanisms to apply in parallel. The
                        output features are equally split up over the heads if concat_heads=True.
            concat_heads - If True, the output of the different heads is concatenated instead of averaged.
            alpha - Negative slope of the LeakyReLU activation.
        """
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = concat_heads
        if self.concat_heads:
            assert c_out % num_heads == 0, "Number of output features must be a multiple of num_heads"
            c_out = c_out // num_heads

        # sub-modules and parameters needed in layer
        self.projection = nn.Linear(c_in, c_out * num_heads)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2 * c_out)) # one per head; weight matrix of MLP
        self.leakyrelu = nn.LeakyReLU(alpha) # need this for node dependency on h_i to attention

        # initialization from the original implementation
        nn.init.xavier_uniform_(self.projection.weight.data, gain=1.414) # gain is factor for LeakyReLU
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

    def forward(self, node_feats, adj_matrix, print_attn_probs=False):
        """
        Inputs:
            node_feats - Input features of the node. Shape: [batch_size, c_in]
            adj_matrix - Adjacency matrix including self-connections. Shape: [batch_size, num_nodes, num_nodes]
            print_attn_probs - If True, the attention weights are printed during the forward pass (for debugging purposes)
        """
        print("Adj matrix shape = \n", adj_matrix.shape)
        batch_size, num_nodes = adj_matrix.size(0), adj_matrix.size(1)

        # apply linear layer and sort nodes by head
        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)

        # Attention logits for each edge needs to be calculated
        # doing this on all possible combinations is expensive
        # => Create a tensor of [W*h_i||W*h_j] with i and j being the indices of all edges
        edges = adj_matrix.nonzero(as_tuple=False) # return indices where adj matrix is non-zero
        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)
        edge_indices_row = edges[:,0] * num_nodes + edges[:,1]
        edge_indices_col = edges[:,0] * num_nodes + edges[:,2]

        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)
        ], dim=-1) # index select returns tensor with node_feats_flat being indexed @ desired positions along dim=0

        #print("a_input shape: \n", a_input.shape)
        #print("self.a shape: \n", self.a.shape)
        
        # calculate the attention MLP output (independent for each head)
        # batch multiplication and then sum across j-th dim
        attn_logits = self.leakyrelu(torch.einsum("ijk,jk->ik", a_input, self.a))
        #print("attn_logits shape: \n", attn_logits.shape)

        # map list of attention values back into a matrix
        attn_matrix = attn_logits.new_zeros(adj_matrix.shape + (self.num_heads,)).fill_(-9e15)
        # n[..., None] means n.unsqueeze(dim=-1)
        # n[None] means n.unsqueeze(dim=1)
        # repeat(): repeat tensor along the specified dimensions
        attn_matrix[adj_matrix[...,None].repeat(1,1,1,self.num_heads) == 1] = attn_logits.reshape(-1)
        # if the nodes are not connected, then the attn_matrix element = 0

        # weighted average of attention
        attn_probs = F.softmax(attn_matrix, dim=2)
        if print_attn_probs:
            print("Attention probs\n", attn_probs.permute(0,3,1,2))
        
        # bijh @ bjhc = bihhc 
        # then  sum along first h axis to get bihc
        node_feats = torch.einsum("bijh,bjhc->bihc", attn_probs, node_feats)

        # if heads should be concatenated, we can do this by reshaping. Otherwise, take mean
        if self.concat_heads:
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:
            node_feats = node_feats.mean(dim=2)

        return node_feats


In [116]:
layer = GATLayer(2, 2, num_heads=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]]) # identity
layer.projection.bias.data = torch.Tensor([0., 0.])
layer.a.data = torch.Tensor([[-0.2, 0.3], [0.1, -0.1]])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix, print_attn_probs=True)

print("Adjacency matrix: \n", adj_matrix)
print("Input features: \n", node_feats)
print("Output features: \n", out_feats)

Adj matrix shape = 
 torch.Size([1, 4, 4])
Attention probs
 tensor([[[[0.5000, 0.5000, 0.0000, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.2500],
          [0.0000, 0.3333, 0.3333, 0.3333],
          [0.0000, 0.3333, 0.3333, 0.3333]],

         [[0.4207, 0.5793, 0.0000, 0.0000],
          [0.1334, 0.1837, 0.2741, 0.4088],
          [0.0000, 0.2120, 0.3162, 0.4718],
          [0.0000, 0.2120, 0.3162, 0.4718]]]])
Adjacency matrix: 
 tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features: 
 tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features: 
 tensor([[[1.0000, 2.1586],
         [3.0000, 4.9167],
         [4.0000, 5.5196],
         [4.0000, 5.5196]]])


### PyTorch Geometric

Implementing graph networks with adjacency matrix is simple and straight-forward but can be computationally expensive for large graphs. Many real-world graphs can reach over 200k nodes, for which adjacency matrix-based implementations fail. There are a lot of optimizations possible when implementing GNNs, and luckily, there exist packages that provide such layers. The most popular packages for PyTorch are PyTorch Geometric and the Deep Graph Library (the latter being actually framework agnostic).

In [118]:
try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric
    import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
     |████████████████████████████████| 7.9 MB 9.8 MB/s eta 0:00:01
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
     |████████████████████████████████| 3.5 MB 10.3 MB/s eta 0:00:01
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
     |████████████████████████████████| 2.5 MB 9.5 MB/s eta 0:00:01
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
     |████████████████████████████████| 751 kB 8.0 MB/s eta 0:00:01


PyTorch Geometric uses a list of index pairs to represent the edges.

In [120]:
gnn_layer_by_name = {
    "GCN": geom_nn.GCNConv,
    "GAT": geom_nn.GATConv,
    "GraphConv": geom_nn.GraphConv
}

# GraphConv is a GCN with a separate weight matrix for the self-connections. The neighbor's messages
# are added instead of average

GraphConv is a GCN with a separate weight matrix for the self-connections. The neighbor's messages are added instead of averaged.

### Experiments on Graph Structures

Tasks on graph-structured data can be grouped into three groups: node-level, edge-level and graph-level. The different levels describe on which level we want to perform classification/regression.

#### Node-level tasks: Semi-supervised node classification
Node-level tasks have the goal to classify nodes in a graph. Usually, we have given a single, large graph with >1000 nodes of which a certain amount of nodes are labeled. We learn to classify those labeled examples during training and try to generalize to the unlabeled nodes.

In [121]:
cora_dataset = torch_geometric.datasets.Planetoid(root=DATASET_PATH, name="Cora")

Processing...
Done!


In [123]:
cora_dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [126]:
class GNNModel(nn.Module):
    def __init__(self, c_in, c_hidden, c_out, num_layers=2, layer_name="GCN", dp_rate=0.1, **kwargs):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of the output features. Usually number of classes in classification
            num_layers - Number of "hidden" graph layers
            layer_name - String of the graph layer to use
            dp_rate - Dropout rate to apply throughout the network
            kwargs - Additional arguments for the graph layer (e.g. number of heads for GAT)
        """
        super().__init__()
        gnn_layer = gnn_layer_by_name[layer_name]

        layers = []
        in_channels, out_channels = c_in, c_hidden
        for _ in range(num_layers-1):
            layers.append(
                gnn_layer(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    **kwargs
                )
            )
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dp_rate))
            in_channels = c_hidden
        layers.append(
            gnn_layer(in_channels=in_channels, out_channels=c_out, **kwargs)
        )

        self.layers = nn.ModuleList(layers)


    def forward(self, x, edge_index):
        """
        Inputs:
            x - Input features per node
            edge_index - List of vertex index pairs representing the edges in the graph 
            (PyTorch geometric notation)
        """
        for l in self.layers:
            # For graph layers, we need to add the "edge_index" tensor as additional input
            # All PyTorch Geometric graph layer inherit the class "MessagePassing", hence
            # we can simply check the class type.
            if isinstance(l, geom_nn.MessagePassing):
                x = l(x, edge_index)
            else:
                x = l(x)

        return x

Good practice in node-level tasks is to create an MLP baseline that is applied to each node independently. This way we can verify whether adding the graph information to the model indeed improves the prediction, or not. It might also be that the features per node are already expressive enough to clearly point towards a specific class. 

In [127]:
class MLPModel(nn.Module):
    def __init__(self, c_in, c_hidden, c_out, num_layers=2, dp_rate=0.2):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of the output features. Usually number of classes in classification
            num_layers - Number of hidden layers
            dp_rate - Dropout rate to apply throughout the network
        """
        super().__init__()
        layers = []
        in_channels, out_channels = c_in, c_hidden
        for _ in range(num_layers-1):
            layers.append(nn.Linear(in_channels, out_channels))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dp_rate))
            in_channels = c_hidden
        layers.append(nn.Linear(in_channels, c_out))

        self.layers = nn.Sequential(*layers)


    def forward(self, x, *args, **kwargs):
        return self.layers(x)

In [128]:
# merge the models into a PyTorch Lightning module which handles the training, validation, and testing
class NodeLevelGNN(pl.LightningModule):
    def __init__(self, model_name, **model_kwargs):
        super().__init__()
        self.save_hyperparameters()

        if "mlp" in model_name.lower():
            self.model = MLPModel(**model_kwargs)
        else:
            self.model = GNNModel(**model_kwargs)
        
        self.loss_module = nn.CrossEntropyLoss()


    # BOILER PLATE FUNCTIONS ==================================================================
    def forward(self, data, mode="train"):
        x, edge_index = data.x, data.edge_index
        x = self.model(x, edge_index)

        # only calculate the loss on nodes corresponding to mask
        if mode == "train":
            mask = data.train_mask
        elif mode == "val":
            mask = data.val_mask
        elif mode == "test":
            mask = data.test_mask
        else:
            assert False, f"Unknown forward mode: {mode}"

        loss = self.loss_module(x[mask], data.y[mask])
        acc = (x[mask].argmax(dim=-1) == data.y[mask]).sum().float() / mask.sum()

        return loss, acc

    def configure_optimizers(self):
        # SGD used here but Adam works well too
        optimizer = optim.SGD(self.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log("val_acc", acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log("test_acc", acc)


In [129]:
# define training function for Lightning module
def train_node_classifier(model_name, dataset, **model_kwargs):
    pl.seed_everything(42)
    node_data_loader = geom_data.DataLoader(dataset, batch_size=1)

    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "NodeLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=200,
                         progress_bar_refresh_rate=0) # 0 because epoch size is 1
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"NodeLevel{model_name}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = NodeLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything()
        model = NodeLevelGNN(model_name=model_name, c_in=dataset.num_node_features, c_out=dataset.num_classes, **model_kwargs)
        trainer.fit(model, node_data_loader, node_data_loader)
        model = NodeLevelGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

    # Test best model on the test set
    test_result = trainer.test(model, node_data_loader, verbose=False)
    batch = next(iter(node_data_loader))
    batch = batch.to(model.device)
    _, train_acc = model.forward(batch, mode="train")
    _, val_acc = model.forward(batch, mode="val")
    result = {"train": train_acc,
              "val": val_acc,
              "test": test_result[0]['test_acc']}
    return model, result

In [130]:
# Small function for printing the test scores
def print_results(result_dict):
    if "train" in result_dict:
        print(f"Train accuracy: {(100.0*result_dict['train']):4.2f}%")
    if "val" in result_dict:
        print(f"Val accuracy:   {(100.0*result_dict['val']):4.2f}%")
    print(f"Test accuracy:  {(100.0*result_dict['test']):4.2f}%")

In [131]:
node_mlp_model, node_mlp_result = train_node_classifier(model_name="MLP",
                                                        dataset=cora_dataset,
                                                        c_hidden=16,
                                                        num_layers=2,
                                                        dp_rate=0.1)

print_results(node_mlp_result)

Global seed set to 42
/home/johnathon/miniconda3/envs/dl2021/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Found pretrained model, loading...


Missing logger folder: ../saved_models/tutorial7/NodeLevelMLP/lightning_logs
/home/johnathon/miniconda3/envs/dl2021/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Train accuracy: 97.14%
Val accuracy:   54.60%
Test accuracy:  60.60%


In [132]:
node_gnn_model, node_gnn_result = train_node_classifier(model_name="GNN",
                                                        layer_name="GCN",
                                                        dataset=cora_dataset,
                                                        c_hidden=16,
                                                        num_layers=2,
                                                        dp_rate=0.1)
print_results(node_gnn_result)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: ../saved_models/tutorial7/NodeLevelGNN/lightning_logs


Found pretrained model, loading...
Train accuracy: 100.00%
Val accuracy:   78.60%
Test accuracy:  82.40%


The GNN model outperforms the MLP by quite a margin. This shows that using the graph information indeed improves our predictions and lets us generalizes better.

The hyperparameters in the model have been chosen to create a relatively small network. This is because the first layer with an input dimension of 1433 can be relatively expensive to perform for large graphs. In general, GNNs can become relatively expensive for very big graphs. This is why such GNNs either have a small hidden size or use a special batching strategy where we sample a connected subgraph of the big, original graph.

#### Edge-level tasks: Link prediction
In some applications, we might have to predict on an edge-level instead of node-level. The most common edge-level task in GNN is link prediction. Link prediction means that given a graph, we want to predict whether there will be/should be an edge between two nodes or not.

The output prediction is usually done by performing a similarity metric on the pair of node features, which should be 1 if there should be a link, and otherwise close to 0. To keep the tutorial short, we will not implement this task ourselves. Nevertheless, there are many good resources out there if you are interested in looking closer at this task. One tutorial [here](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py).

#### Graph Level Tasks: Graph Classification
The goal is to classify an entire graph instead of single nodes or edges. Therefore, we are also given a dataset of multiple graphs that we need to classify based on some structural graph properties. The most common task for graph classification is molecular property prediction, in which molecules are represented as graphs. Each atom is linked to a node, and edges in the graph are the bonds between atoms.

In [135]:
tu_dataset = torch_geometric.datasets.TUDataset(root=DATASET_PATH, name="MUTAG")

print("Data object:", tu_dataset.data)
print("Length:", len(tu_dataset))
print(f"Average label: {tu_dataset.data.y.float().mean().item():4.2f}")

Data object: Data(x=[3371, 7], edge_index=[2, 7442], edge_attr=[7442, 4], y=[188])
Length: 188
Average label: 0.66


 It happens quite often that graph datasets are very imbalanced, hence checking the class balance is always a good thing to do.

In [136]:
torch.manual_seed(42)
tu_dataset.shuffle()
train_dataset = tu_dataset[:150]
test_dataset = tu_dataset[150:]

When using a data loader, we encounter a problem with batching $N$ graphs. Each graph in the batch can have a different number of nodes and edges, and hence we would require a lot of padding to obtain a single tensor. Torch geometric uses a different, more efficient approach: we can view the $N$ graphs in a batch as a single large graph with concatenated node and edge list. As there is no edge between the $N$ graphs, running GNN layers on the large graph gives us the same output as running the GNN on each graph separately. <br>

Adjacency matrix is zero for any nodes that come from two different graphs, and otherwise according to the adjacency matrix of the individual graph.

In [137]:
graph_train_loader = geom_data.DataLoader(train_dataset, batch_size=64, shuffle=True)
graph_val_loader = geom_data.DataLoader(test_dataset, batch_size=64) # Additional loader if you want to change to a larger dataset
graph_test_loader = geom_data.DataLoader(test_dataset, batch_size=64)

In [138]:
batch = next(iter(graph_test_loader))
print("Batch:", batch)
print("Labels:", batch.y[:10])
print("Batch indices:", batch.batch[:40])

Batch: DataBatch(edge_index=[2, 1512], x=[687, 7], edge_attr=[1512, 4], y=[38], batch=[687], ptr=[39])
Labels: tensor([1, 1, 1, 0, 0, 0, 1, 1, 1, 0])
Batch indices: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2])


In [144]:
class GraphGNNModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, dp_rate_linear=0.5, **kwargs):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of output features (usually number of classes)
            dp_rate_linear - Dropout rate before the linear layer (usually much higher than inside the GNN)
            kwargs - Additional arguments for the GNNModel object
        """
        super().__init__()
        self.GNN = GNNModel(c_in=c_in,
                            c_hidden=c_hidden,
                            c_out=c_hidden, # Not our prediction output yet!
                            **kwargs)
        self.head = nn.Sequential(
            nn.Dropout(dp_rate_linear),
            nn.Linear(c_hidden, c_out)
        )

    def forward(self, x, edge_index, batch_idx):
        """
        Inputs:
            x - Input features per node
            edge_index - List of vertex index pairs representing the edges in the graph (PyTorch geometric notation)
            batch_idx - Index of batch element for each node
        """
        x = self.GNN(x, edge_index)
        x = geom_nn.global_mean_pool(x, batch_idx) # Average pooling
        x = self.head(x)
        return x

In [145]:
class GraphLevelGNN(pl.LightningModule):

    def __init__(self, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()

        self.model = GraphGNNModel(**model_kwargs)
        self.loss_module = nn.BCEWithLogitsLoss() if self.hparams.c_out == 1 else nn.CrossEntropyLoss()

    def forward(self, data, mode="train"):
        x, edge_index, batch_idx = data.x, data.edge_index, data.batch
        x = self.model(x, edge_index, batch_idx)
        x = x.squeeze(dim=-1)

        if self.hparams.c_out == 1:
            preds = (x > 0).float()
            data.y = data.y.float()
        else:
            preds = x.argmax(dim=-1)
        loss = self.loss_module(x, data.y)
        acc = (preds == data.y).sum().float() / preds.shape[0]
        return loss, acc

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=1e-2, weight_decay=0.0) # High lr because of small dataset and small model
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

In [140]:
def train_graph_classifier(model_name, **model_kwargs):
    pl.seed_everything(42)

    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "GraphLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=500,
                         progress_bar_refresh_rate=0)
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"GraphLevel{model_name}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = GraphLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42)
        model = GraphLevelGNN(c_in=tu_dataset.num_node_features,
                              c_out=1 if tu_dataset.num_classes==2 else tu_dataset.num_classes,
                              **model_kwargs)
        trainer.fit(model, graph_train_loader, graph_val_loader)
        model = GraphLevelGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    # Test best model on validation and test set
    train_result = trainer.test(model, graph_train_loader, verbose=False)
    test_result = trainer.test(model, graph_test_loader, verbose=False)
    result = {"test": test_result[0]['test_acc'], "train": train_result[0]['test_acc']}
    return model, result



In [146]:
model, result = train_graph_classifier(model_name="GraphConv",
                                       c_hidden=256,
                                       layer_name="GraphConv",
                                       num_layers=3,
                                       dp_rate_linear=0.5,
                                       dp_rate=0.0)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: ../saved_models/tutorial7/GraphLevelGraphConv/lightning_logs
/home/johnathon/miniconda3/envs/dl2021/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:376: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/home/johnathon/miniconda3/envs/dl2021/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Found pretrained model, loading...


In [147]:
print(f"Train performance: {100.0*result['train']:4.2f}%")
print(f"Test performance:  {100.0*result['test']:4.2f}%")

Train performance: 93.28%
Test performance:  92.11%
